In [1]:
!pip install nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
data = pd.read_csv('azureml://subscriptions/512a781e-d15a-4734-adca-96ec827531cb/resourcegroups/xwang3306-rg/workspaces/DVA_PROJECT/datastores/workspaceblobstore/paths/UI/2024-10-16_040004_UTC/consolidated_cleaned_neiss_2014_2023.csv')

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import numpy as np
data['#_prod']=np.nan
data.loc[((data['Product_1']>0)), '#_prod'] = 1
data.loc[((data['Product_1']>0) & (data['Product_2']>0)), '#_prod'] = 2
data.loc[((data['Product_1']>0) & (data['Product_2']>0) & (data['Product_3']>0)), '#_prod'] = 3

In [4]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [5]:
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts()

0    3085714
1     378362
Name: Disposition_recode_2, dtype: int64

In [6]:
def remove_after_dx(narrative):
  if isinstance(narrative, str):
    parts = narrative.split("DX", 1)
    if len(parts) > 1:
      return parts[0]
    else:
      return narrative  # No "DX:" found, return the original string
  else:
    return narrative  # Not a string, return as is


replace_list=['ANKLE', 'ARM', 'BODY_PART', 'CHEST', 'CONTUSION', 'CUT', 'EAR', 'ELBOW', 'EYE', 'FACE', 'FINGER', 'FOOT', 'FOREHEAD', 'FRACTURE', 'FX', 'HAND', 'HEAD', 'HIP', 'KNEE', 'LAC', 'LACERATION', 'LEG', 'LOC', 'LOSE', 'NECK', 'PAIN', 'SHOULDER', 
'SPRAIN', 'STRAIN', 'SWELL', 'THUMB', 'TOE', 'WRIST','ABRASION', 'ACHE', 'BREAK', 'BURN', 'CHIN', 'CUT', 'ER', 'FALL', 'FRACTURE', 'FX', 'HIT', 'INJURY', 'LACERATION', 'LIP', 'LOSE', 'LOC', 'MOUTH', 'NOSE', 'PAIN', 'RIB', 'SCALP', 'SPRAIN', 'STRAIN', 'SWELL', 'TOE', 'TWIST', 'WRIST']

for i in replace_list:
  data['Narrative'] = data['Narrative'].str.replace(i, '')

data['Narrative'] = data['Narrative'].apply(remove_after_dx)
data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace('JURED', '')
data['Narrative'] = data['Narrative'].str.replace('URED', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')
data['Narrative'] = data['Narrative'].str.replace(' LT ', '')


In [7]:
core_col=['Disposition_recode_2','Age', 'Sex', 'Race','Location','Hispanic', 'Body_Part','Product_1' ,'Alcohol', 'Drug','Narrative']
data_core=data[core_col]


In [8]:
df_bad=data_core[data_core['Disposition_recode_2']==1]
df_good=data_core[data_core['Disposition_recode_2']==0]

In [12]:
data_good_sample=df_good.sample(frac=0.1).reset_index(drop=True)
data_bad_sample=df_bad.sample(frac=0.8).reset_index(drop=True)
data_core_sample=pd.concat([data_good_sample,data_bad_sample]).reset_index(drop=True)

In [14]:
data_core_sample=data_core_sample.sample(frac=1).reset_index(drop=True)
data_core_sample

,Disposition_recode_2,Age,Sex,Race,#_prod,Location,Hispanic,Body_Part,Product_1,Alcohol,Drug,Narrative
0,1,35,2,1,1.0,0,2.0,36,1842,0.0,0.0,35 SPED DOWN A STAIR ON THE WAY TO WALK H DOG...
1,0,13,2,3,1.0,1,0.0,93,679,0.0,0.0,13PT WAS JUMPING ON THE COUCH WHEN SHE STUBBED...
2,1,76,2,1,1.0,1,2.0,79,1842,0.0,0.0,76 FALL DOWN A FLIGHT OF STAIRS GOING INTO THE...
3,1,66,1,1,2.0,1,0.0,31,5040,0.0,0.0,66 WENT FOR A BICYCLE RIDE 1130 WHEN HE GOT ...
4,1,66,2,0,1.0,1,0.0,79,649,0.0,0.0,66 FALL OFF THE TOILET AND LANDED ONTO D
...,...,...,...,...,...,...,...,...,...,...,...,...
611256,0,10,1,0,1.0,1,0.0,94,1807,0.0,0.0,10 SPED AND FALL ON WET KITCHEN FLOOR
611257,1,92,2,1,1.0,1,0.0,79,1807,0.0,0.0,92 WAS FOUND ON FLOOR AT HOME AFT A
611258,0,27,1,2,1.0,9,2.0,37,1205,0.0,0.0,27 S WAS PLAYING BASKETBALL LAST NIGHT JUMPED...
611259,0,61,2,0,1.0,5,0.0,30,611,0.0,0.0,61 FALL IN THE SHOW AT THE DETOX CENT AND SUST...


In [15]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


corpus = data_core_sample['Narrative'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

data_core_sample['Processed_Narrative'] = corpus.apply(preprocess_text)

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer(max_features=140, stop_words='english')

# Fit and transform the processed text
tfidf_matrix = vectorizer.fit_transform(data_core_sample['Processed_Narrative'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenate the TF-IDF features with your existing data
data_ready = pd.concat([data_core_sample, tfidf_df], axis=1)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
#data_ready = pd.concat([data_core_sample, tfidf_df], axis=1)
#list(data_ready.columns)

In [17]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Separate features (X) and target variable (y)
X = data_ready.drop(['Disposition_recode_2','Narrative','Processed_Narrative'], axis=1)  # ,'Processed_Narrative'] Replace 'target_variable' with your target column name
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))
X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    #'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    #'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(objective='binary:logistic', use_label_encoder=False), 
           {'xgboost__learning_rate': [0.1, 0.01, 0.001],            # Step size shrinkage to prevent overfitting
            'xgboost__max_depth': [3, 5, 7],                         # Maximum depth of each tree
            #'xgboost__n_estimators': [100, 200, 300],                # Number of boosting rounds
            #'xgboost__min_child_weight': [1, 3, 5],                  # Minimum sum of instance weight needed in a child
            #'xgboost__subsample': [0.6, 0.8,0.9],                   # Subsample ratio of the training instances
            #'xgboost__colsample_bytree': [0.6, 0.8, 0.9],            # Subsample ratio of columns when constructing each tree
            #'xgboost__gamma': [0.1, 0.5, 1,3],                     # Minimum loss reduction required for a split
            #'xgboost__reg_alpha': [0.01, 0.1,0.5,1],                    # L1 regularization term on weights
            #'xgboost__reg_lambda': [1, 1.5, 2],                      # L2 regularization term on weights
            #'xgboost__scale_pos_weight': [1, 5, 10]
            }),
    #'Logistic_Regression': (LogisticRegression(solver='liblinear'), {'logistic_regression__C': [0.1, 1, 10]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}),
    #'NN': (MLPClassifier(max_iter=1000), {'nn__hidden_layer_sizes': [(10,), (50,), (100,)], 'nn__activation': ['relu', 'tanh']}),
}

# Initialize list to store predictions from each model
ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='f1_weighted',n_jobs=-1)
    grid_search.fit(X_train_final, y_train_final)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# Implement ensemble method (majority voting for predictions, average for probabilities)
ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Average the probabilities for the ensemble AUC
ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# Evaluate the ensemble model using accuracy, F1-score, and AUC
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
ensemble_f1 = f1_score(y_test, ensemble_pred_final)
ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1 Score:", ensemble_f1)
print("Ensemble AUC:", ensemble_auc)
print(classification_report(y_test, ensemble_pred_final))

Training XGBoost...
[03:40:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:40:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:45:04] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:40:37] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like t

/tmp/ipykernel_18638/3370004319.py:94: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()


In [ ]:
xxx

In [ ]:
pip install imblearn

In [ ]:
xxx